In [2]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

# 이미지 전처리를 위한 transform 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),  # 이미지를 PyTorch Tensor로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 정규화
])

# 학습 및 검증 데이터셋 로드
train_dataset = datasets.ImageFolder(root='datasets/train', transform=transform)
val_dataset = datasets.ImageFolder(root='datasets/val', transform=transform)

# DataLoader 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [5]:
from torchvision import models

num_classes=len(train_dataset.classes)

# 사전 훈련된 ResNet 모델 로드
model = models.resnet18(pretrained=True)

# 마지막 계층을 현재 데이터셋에 맞게 변경
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, num_classes)  # num_classes는 데이터셋에 따라 다름

# GPU 사용 설정
device = torch.device("mps")
print(device)
model = model.to(device)

/Users/everyshare/anaconda3/envs/Location_Classification_model/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/everyshare/anaconda3/envs/Location_Classification_model/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


mps


In [6]:
import torch.optim as optim
from torch.optim import lr_scheduler

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)  # 학습률 조정

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        scheduler.step()

        epoch_loss = running_loss / len(train_dataset)
        epoch_acc = running_corrects.float() / len(train_dataset)

        print(f'Epoch {epoch}/{num_epochs - 1} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

In [10]:
train_model(model,criterion,optimizer,scheduler)

Epoch 0/24 Loss: 0.0325 Acc: 0.9873
Epoch 1/24 Loss: 0.0187 Acc: 0.9932
Epoch 2/24 Loss: 0.0116 Acc: 0.9965
Epoch 3/24 Loss: 0.0097 Acc: 0.9972
Epoch 4/24 Loss: 0.0073 Acc: 0.9978
Epoch 5/24 Loss: 0.0050 Acc: 0.9987
Epoch 6/24 Loss: 0.0035 Acc: 0.9990
Epoch 7/24 Loss: 0.0027 Acc: 0.9994
Epoch 8/24 Loss: 0.0034 Acc: 0.9991
Epoch 9/24 Loss: 0.0026 Acc: 0.9994
Epoch 10/24 Loss: 0.0033 Acc: 0.9993
Epoch 11/24 Loss: 0.0026 Acc: 0.9994
Epoch 12/24 Loss: 0.0024 Acc: 0.9996
Epoch 13/24 Loss: 0.0026 Acc: 0.9995
Epoch 14/24 Loss: 0.0023 Acc: 0.9997
Epoch 15/24 Loss: 0.0024 Acc: 0.9995
Epoch 16/24 Loss: 0.0020 Acc: 0.9997
Epoch 17/24 Loss: 0.0027 Acc: 0.9993
Epoch 18/24 Loss: 0.0023 Acc: 0.9995
Epoch 19/24 Loss: 0.0020 Acc: 0.9997
Epoch 20/24 Loss: 0.0033 Acc: 0.9991
Epoch 21/24 Loss: 0.0027 Acc: 0.9995
Epoch 22/24 Loss: 0.0024 Acc: 0.9998
Epoch 23/24 Loss: 0.0022 Acc: 0.9998
Epoch 24/24 Loss: 0.0018 Acc: 0.9998


In [13]:
def validate_model(model, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

    total_loss = running_loss / len(val_dataset)
    total_acc = running_corrects.float() / len(val_dataset)
    print(f'Validation Loss: {total_loss:.4f} Acc: {total_acc:.4f}')


In [14]:
validate_model(model,criterion)

Validation Loss: 0.0307 Acc: 0.9888


In [16]:
model_path='model_weights.pth'
torch.save(model.state_dict(), model_path)

In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(train_history, val_history, title='Loss'):
    plt.figure()
    plt.plot(train_history, label='Train')
    plt.plot(val_history, label='Validation')
    plt.title(title)
    plt.legend()
    plt.show()

# 예를 들어, 손실과 정확도의 변화를 시각화할 수 있습니다.
# plot_training_history(train_losses, val_losses, 'Loss')
# plot_training_history(train_accuracies, val_accuracies, 'Accuracy')


In [25]:
# 모델을 평가 모드로 설정하고 이미지에 대한 예측 실행, 라벨 반환
import torch.nn.functional as F
# 모델을 평가 모드로 설정하고 이미지에 대한 예측 실행
def predict_image(model, image, idx_to_class):
    model.eval()  # 모델을 평가 모드로 설정
    image = image.to(device)
    
    with torch.no_grad():
        outputs = model(image)
        # 확률을 얻기 위해 softmax 함수 적용
        probabilities = F.softmax(outputs, dim=1)
        # 최대 확률을 가진 클래스의 인덱스와 확률 값 추출
        top_p, top_class = probabilities.topk(1, dim=1)
        idx = top_class.item()
        pred_class = idx_to_class[idx]
        pred_prob = top_p.item()
        
    return pred_class, pred_prob

In [18]:
from PIL import Image
def load_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image=Image.open(image_path).convert('RGB')
    image=transform(image).unsqueeze(0)
    return image

In [27]:
image_path='datasets/test3.jpg'
image=load_image(image_path)

In [20]:
idx_to_class={v: k for k,v in train_dataset.class_to_idx.items()}
idx_to_class

{0: 'cat', 1: 'dog'}

In [28]:
predicted_class, predicted_prob = predict_image(model, image, idx_to_class)

# 결과 출력
print(f"Predicted class: {predicted_class}, Probability: {predicted_prob:.4f}")

Predicted class: dog, Probability: 0.9768
